In [343]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import pickle

print('TensorFlow version: {}'.format(tf.__version__))

TensorFlow version: 2.12.0


In [344]:
from db_actions import DatabaseActions as db

data = db.get_parsed()
print(data[7][0])

Хороший ресторан. Приятно отдыхать. 


In [345]:
dataset = [[data[0][0], pickle.loads(bytes.fromhex(data[0][1])), data[0][2]]]
for i in range(1, len(data)):
        dataset.append([data[i][0], pickle.loads(bytes.fromhex(data[i][1])), data[i][2]])

print(dataset[0][2])

# 0 -- text
# 1 -- vector
# 2 -- is_fake

0


In [346]:
fake_len = 100
true_len = 100    

texts = []
x_train_t = []
y_train_t = []
for i in range(len(dataset)):
    if fake_len <= 0:
        break
    if dataset[i][2] == 1:
        x_train_t.append(dataset[i][1])
        y_train_t.append(dataset[i][2])
        texts.append(dataset[i][0])
        fake_len -= 1


for i in range(len(dataset)):
    if true_len <= 0:
        break
    if dataset[i][2] == 0:
        x_train_t.append(dataset[i][1])
        y_train_t.append(dataset[i][2])
        texts.append(dataset[i][0])
        true_len -= 1


In [347]:

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

vectors_tmp = []
for record in dataset:
    vectors_tmp.append(record[1])

# make numpy array vectors
vectors = np.array(vectors_tmp)


print(vectors.shape)


(711, 768)


In [348]:
# print max value in vector in vectors by absolute value
def max_abs_value(vectors):
    return max(abs(np.min(vectors)), np.max(vectors))

print(max_abs_value(x_train_t), end= '\n' + '~' * 80 + '\n')

x_train_t2 = vectors / max_abs_value(x_train_t)

x_train_t2 += 1
x_train = x_train_t / max_abs_value(x_train_t) * (1 - 1 / 10 ** 5)

print(max_abs_value(x_train))


3.6503556
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0.99999


In [349]:

# make numpy array vectors
y_train_t2 = np.array(y_train_t)
y_train = np.array(y_train_t).astype(bool)

print(y_train[1:10])

print(x_train.shape)
print(y_train.shape)

[ True  True  True  True  True  True  True  True  True]
(200, 768)
(200,)


In [350]:
print(x_train)
print(y_train)

print(x_train.shape)
print(y_train.shape)

[[-0.1796261  -0.3177063   0.18833625 ...  0.2663538  -0.21756333
   0.04852504]
 [-0.17986612 -0.2641443   0.13951811 ...  0.41062406 -0.13013798
   0.08484519]
 [-0.12720522 -0.38686308  0.11604431 ...  0.35385123 -0.06880906
   0.07288557]
 ...
 [-0.16156684 -0.3783506   0.12312912 ...  0.3832249  -0.12081898
   0.06430423]
 [-0.1941543  -0.22815195 -0.03702387 ...  0.384088   -0.05068755
   0.09702554]
 [-0.15354572 -0.20752871  0.2105557  ...  0.32855862 -0.175106
   0.07323228]]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True

In [359]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Conv1D, GlobalMaxPooling1D, Attention, Reshape

model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024 * 4, activation='elu', input_shape=(768,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024 * 3, activation='elu'),
        tf.keras.layers.Dense(1024 * 3, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024 * 2, activation='elu'),
        tf.keras.layers.Dense(1024 * 2, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='elu'),
        tf.keras.layers.Dense(1024, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='elu'),
        tf.keras.layers.Dense(256, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='elu'),
        tf.keras.layers.Dense(64, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])


# # Define the model architecture
# model = Sequential()
# model.add(Embedding(input_dim=1, output_dim=300, input_length=768))  # Embedding layer for word embeddings
# model.add(Conv1D(256, 5, activation='relu', padding='same'))  # Convolutional layer with 256 filters and kernel size of 5
# model.add(GlobalMaxPooling1D())  # Global max pooling layer
# # model.add(Reshape((256, 1)))  # Reshape the output to make it three-dimensional
# # model.add(LSTM(512, return_sequences=False))  # LSTM layer with 512 units
# # model.add(Attention())  # Attention layer to focus on important parts of the sequence
# model.add(Dense(256, activation='relu'))  # Dense layer with 256 neurons
# model.add(Dropout(0.5))  # Dropout layer for regularization
# model.add(Dense(128, activation='relu'))  # Additional dense layer with 128 neurons
# model.add(Dropout(0.5))  # Additional dropout layer
# model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# model.summary()


# Train the model
model.fit(x_train, y_train, epochs=15, verbose=1)



Epoch 1/15
7/7 [==============================] - 3s 210ms/step - loss: 1.7056 - accuracy: 0.4800
Epoch 2/15
7/7 [==============================] - 1s 208ms/step - loss: 2.7382 - accuracy: 0.5400
Epoch 3/15
7/7 [==============================] - 1s 206ms/step - loss: 2.4565 - accuracy: 0.5650
Epoch 4/15
7/7 [==============================] - 2s 216ms/step - loss: 2.0109 - accuracy: 0.6050
Epoch 5/15
7/7 [==============================] - 2s 224ms/step - loss: 1.7561 - accuracy: 0.5800
Epoch 6/15
7/7 [==============================] - 2s 211ms/step - loss: 1.3977 - accuracy: 0.5900
Epoch 7/15
7/7 [==============================] - 1s 209ms/step - loss: 1.1029 - accuracy: 0.5950
Epoch 8/15
7/7 [==============================] - 1s 211ms/step - loss: 1.2090 - accuracy: 0.5000
Epoch 9/15
7/7 [==============================] - 1s 209ms/step - loss: 0.9120 - accuracy: 0.5500
Epoch 10/15
7/7 [==============================] - 1s 209ms/step - loss: 0.8186 - accuracy: 0.5800
Epoch 11/15
7/7 [==

In [364]:
for i in range(95, 105):
    print(i, y_train[i], texts[i], sep= '\t', end='\t')
    print()


res = model.predict(x_train[0:200])
dif = np.sum(res) / np.size(res)

print(dif)

for i in range(95, 105):
    print('(', i, ') ', res[i] > dif, end='\t')

95	True	Интересный интерьер: разрозненные и разноуровневые места и столики где посидеть выпить/поесть, большое свободное пространство пройтись и посмотреть на публику/анутраж, потанцевать под музычку. В летнее время особенно классно во дворике сидеть и пить рислинг А когда похолодает - зайти и поглядеть на камин
А еще во время вечеринок ночью можно отведать стритфуд в отдельном поинте - этого таааак порой не хватает в других заведениях	
96	True	Отмечали свадьбу племянницы в небольшом, но очень уютном зале. Салаты, конопешечки, закусочки в оригинальной подаче, все очень вкусно. Обслуживание на высоте, официанты внимательные, бокал всегда был наполнен: Впереди лето, обязательно и с большим удовольствием вернусь в это заведение посетив основной зал. 	
97	True	Отмечали юбилей подруги. Отличное кафе. Очень понравилась сервировка - организация праздничного стола. Официанты молодцы! Тихо, незаметно, всё вовремя. Мне здесь было очень уютно. Спасибо. 	
98	True	Праздновали в ресторане свадьбу, о

In [367]:
import os

model.summary()
model.save('model-31052023')

Model: "sequential_85"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_414 (Dense)           (None, 4096)              3149824   
                                                                 
 dropout_106 (Dropout)       (None, 4096)              0         
                                                                 
 dense_415 (Dense)           (None, 3072)              12585984  
                                                                 
 dense_416 (Dense)           (None, 3072)              9440256   
                                                                 
 dropout_107 (Dropout)       (None, 3072)              0         
                                                                 
 dense_417 (Dense)           (None, 2048)              6293504   
                                                                 
 dense_418 (Dense)           (None, 2048)            

2023-05-31 11:08:42.730749: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,4096]
	 [[{{node inputs}}]]
2023-05-31 11:08:42.757491: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,3072]
	 [[{{node inputs}}]]
2023-05-31 11:08:42.775465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,2048]
	 [[{{node inputs}}]]
2023-05-31 11

INFO:tensorflow:Assets written to: model-31052023/assets


INFO:tensorflow:Assets written to: model-31052023/assets
